Title: RP- Spatial Accessibility of COVID-19 Healthcare Resources in Illinois
---

### Reproduction

**Reproduction of**: Rapidly measuring spatial accessibility of COVID-19 healthcare resources: a case study of Illinois, USA

Original study *by* Kang, J. Y., A. Michels, F. Lyu, Shaohua Wang, N. Agbodo, V. L. Freeman, and Shaowen Wang. 2020. Rapidly measuring spatial accessibility of COVID-19 healthcare resources: a case study of Illinois, USA. International Journal of Health Geographics 19 (1):1–17. DOI:[10.1186/s12942-020-00229-x](https://ij-healthgeographics.biomedcentral.com/articles/10.1186/s12942-020-00229-x).

Reproduction Authors: Joe Holler, Kufre Udoh, Derrick Burt, Drew An-Pham, & Spring '21 Middlebury Geog 0323.

Reproduction Materials Available at: [github.com/HEGSRR/RPr-Kang-2020](https://github.com/HEGSRR/RPr-Kang-2020)

Created: `8 Jun 2021`
Revised: `26 Feb 2023`


## Computational Environment
The original study was conducted using an undocumented version of Python on the University of Illinois Urbana Champaign CyberGISX server.
The current version of the study is designed to run on the CyberGISX Python 3 environment using Python version 3.8.12.
See `environment.yml` for more details on the library versions.
Key packages include the following:

In [ ]:
# report python version and install required packages
# switch if statement from False to True to install packages with Conda
if False:
    !python -V
    !conda install -c conda-forge osmnx=1.1.2 -y
    !conda env update --file ../environment/environment.yml -q

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import re
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import folium
import itertools
import os
import time
import warnings
from scipy import stats
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

### Set working directory

Set the working directory to the root of the research compendium.

In [ ]:
# if the working directory is the 'code' folder, move up two folders to the compendium root
if os.path.basename(os.getcwd()) == 'code':
    # This line moves the directory up two folders 
    os.chdir('../../')
print('The working directory is', os.getcwd())

## Data
To perform the ESFCA method, three types of data are required, as follows: (1) road network, (2) population, and (3) hospital information. The road network can be obtained from the [OpenStreetMap Python Library, called OSMNX](https://github.com/gboeing/osmnx). The population data is available on the [American Community Survey](https://data.census.gov/cedsci/deeplinks?url=https%3A%2F%2Ffactfinder.census.gov%2F&tid=GOVSTIMESERIES.CG00ORG01). Lastly, hosptial information is also publically available on the [Homelanad Infrastructure Foundation-Level Data](https://hifld-geoplatform.opendata.arcgis.com/datasets/hospitals?geometry=-94.504%2C40.632%2C-80.980%2C43.486).

### Population and COVID-19 Cases Data by County

In [ ]:
# Load data for at risk population
atrisk_data = gpd.read_file('./data/raw/public/PopData/Chicago_Tract.shp')
atrisk_data.head()

In [ ]:
# Load data for covid cases
covid_data = gpd.read_file('./data/raw/public/PopData/Chicago_ZIPCODE.shp')
covid_data['cases'] = covid_data['cases']
covid_data.head()

### Hospital Data

Note that 999 is treated as a "NULL"/"NA" so these hospitals are filtered out. This data contains the number of ICU beds and ventilators at each hospital.

In [ ]:
# Load data for hospitals
hospitals = gpd.read_file('./data/raw/public/HospitalData/Chicago_Hospital_Info.shp')
hospitals.head()

### Generate and Plot Map of Hospitals

In [ ]:
# Plot hospitals
m = folium.Map(location=[41.85, -87.65], tiles='cartodbpositron', zoom_start=10)

for i in range(0, len(hospitals)):
    folium.CircleMarker(
      location=[hospitals.iloc[i]['Y'], hospitals.iloc[i]['X']],
      popup="{}{}\n{}{}\n{}{}".format('Hospital Name: ',hospitals.iloc[i]['Hospital'],
                                      'ICU Beds: ',hospitals.iloc[i]['Adult ICU'],
                                      'Ventilators: ', hospitals.iloc[i]['Total Vent']),
      radius=5,
      color='grey',
      fill=True,
      fill_opacity=0.6,
      legend_name = 'Hospitals'
    ).add_to(m)
legend_html =   '''<div style="position: fixed; width: 20%; heigh: auto;
                            bottom: 10px; left: 10px;
                            solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Legend<br>'''

m

### Load and Plot Hexagon Grid (500-meter resolution)

In [ ]:
# Load grid file and plot
grid_file = gpd.read_file('./data/raw/public/GridFile/Chicago_Grid.shp')
grid_file = grid_file.drop(columns=["left","right","top","bottom"])
grid_file.plot(figsize=(8,8))

### Load Street Network

Load a street network for either the extent of Chicago, Illinois (provided with the original research compendium) or for the extent of Chicago, Illinois expanded with a buffer distance of 15 miles (24140.2 meters).

In [ ]:
%%time
buffered = True
# change buffered to True to load an expanded (buffered) street network

if not buffered:
    print("Loading original unbuffered Chicago road network from data/raw/public/Chicago_Network.graphml. Please wait...", flush=True)
    G = ox.load_graphml('./data/raw/public/Chicago_Network.graphml') 

# add code to download from OSF @ https://osf.io/download/z8ery/ 
elif not os.path.exists("./data/raw/private/Chicago_Network_Buffer.graphml"):
    print("Loading buffered Chicago road network from OpenStreetMap. Please wait... runtime may exceed 9min...", flush=True)
    G = ox.graph_from_place('Chicago', network_type='drive', buffer_dist=24140.2) 
    print("Saving Chicago road network to raw/private/Chicago_Network_Buffer.graphml. Please wait...", flush=True)
    ox.save_graphml(G, './data/raw/private/Chicago_Network_Buffer.graphml')
    print("Data saved.")

else:
    print("Loading buffered Chicago road network from raw/private/Chicago_Network_Buffer.graphml. Please wait...", flush=True)
    G = ox.load_graphml('./data/raw/private/Chicago_Network_Buffer.graphml') 
    print("Data loaded.")

### Plot the Road Network

In [ ]:
# %%time
ox.plot_graph(G, node_size = 1, bgcolor = 'white', node_color = 'black', edge_color = "#333333", node_alpha = 0.5, edge_linewidth = 0.5)

## Data Processing

In this section, raw data is processed in preparation for the accessibility analysis. Functions are first defined and then called to process the data.

### network_setting

Cleans the OSMNX network to work better with drive-time analysis.

First, we remove all nodes with 0 outdegree because any hospital assigned to such a node would be unreachable from everywhere. Next, we remove small (under 10 node) *strongly connected components* to reduce erroneously small ego-centric networks. Lastly, we ensure that the max speed is set and in the correct units before calculating time.

Args:

* network: OSMNX network for the spatial extent of interest

Returns:

* OSMNX network: cleaned OSMNX network for the spatial extent

In [ ]:
def network_setting(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg ==0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg ==0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component)<10:
            for node in component:
                _nodes_removed+=1
                network.remove_node(node)
    for u, v, k, data in tqdm(G.edges(data=True, keys=True),position=0):
        if 'maxspeed' in data.keys():
            speed_type = type(data['maxspeed'])
            if (speed_type==str):
                # Add in try/except blocks to catch maxspeed formats that don't fit Kang et al's cases
                try:
                    if len(data['maxspeed'].split(','))==2:
                        data['maxspeed_fix']=float(data['maxspeed'].split(',')[0])                  
                    elif data['maxspeed']=='signals':
                        data['maxspeed_fix']=35.0 # Drive speed setting as 35 miles
                    else:
                        data['maxspeed_fix']=float(data['maxspeed'].split()[0])
                except:
                    data['maxspeed_fix']=35.0 # Miles
            else:
                try:
                    data['maxspeed_fix']=float(data['maxspeed'][0].split()[0])
                except:
                    data['maxspeed_fix']=35.0 # Miles
        else:
            data['maxspeed_fix']= 35.0 # Miles
        data['maxspeed_meters'] = data['maxspeed_fix']*26.8223 # Convert mile to meter
        data['time'] = float(data['length'])/ data['maxspeed_meters']
    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed/float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))
    return(network)

#### Pre Process Street network

First, check speed limit attribute data and total number of edges.

In [ ]:
%%time
# Turn edges into geodataframe
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Get unique counts of road segments for each speed limit
print('Total segments: ', len(edges))
print('No speed data: ', edges['maxspeed'].isna().sum())
print(edges['maxspeed'].value_counts())

Second, use the network_setting function to simplify the network and process speed limit data into driving time.

In [ ]:
%%time
# G, hospitals, grid_file, pop_data = file_import (population_dropdown.value, place_dropdown.value)
G = network_setting(G)
# Create point geometries for each node in the graph, to make constructing catchment area polygons easier
for node, data in G.nodes(data=True):
    data['geometry']=Point(data['x'], data['y'])
# Modify code to react to processor dropdown (got rid of file_import function)

Third, review speed limit attribute data and total number of edges.

In [ ]:
%%time
# Turn edges into geodataframe
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Get unique counts of road segments for each speed limit
print('Total segments: ', len(edges))
print('No speed data: ', edges['maxspeed_fix'].isna().sum())
print('Speed\tn')
print(edges['maxspeed_fix'].value_counts())

### hospital_setting

Finds the nearest OSMNX node for each hospital.

Args:

* hospital: GeoDataFrame of hospitals
* G: OSMNX network

Returns:

* GeoDataFrame of hospitals with info on nearest OSMNX node

In [ ]:
def hospital_setting(hospitals, G):
    # Create an empty column 
    hospitals['nearest_osm']=None
    # Append the neaerest osm column with each hospitals neaerest osm node
    for i in tqdm(hospitals.index, desc="Find the nearest osm from hospitals", position=0):
        hospitals['nearest_osm'][i] = ox.get_nearest_node(G, [hospitals['Y'][i], hospitals['X'][i]], method='euclidean') # find the nearest node from hospital location
    print ('hospital setting is done')
    return(hospitals)

### Join Hospitals to Street Network

Use the hospital_setting function to attach hospitals to the nearest street network node

In [ ]:
%%time
hospitals = hospital_setting(hospitals, G)

### pop_centroid

Converts polygon data to centroid points. 

Args:

* pop_data: a GeodataFrame
* pop_type: a string, either "pop" for general population or "covid" for COVID-19 case data

Returns:

* GeoDataFrame of centroids with population data

In [ ]:
# To estimate the centroids of census tract / county
def pop_centroid (pop_data, pop_type):
    pop_data = pop_data.to_crs({'init': 'epsg:4326'})
    # If pop is selected in dropdown, select at risk pop where population is greater than 0
    if pop_type =="pop":
        pop_data=pop_data[pop_data['OverFifty']>=0] 
    # If covid is selected in dropdown, select where covid cases are greater than 0
    if pop_type =="covid":
        pop_data=pop_data[pop_data['cases']>=0]
    pop_cent = pop_data.centroid # it make the polygon to the point without any other information
    # Convert to gdf
    pop_centroid = gpd.GeoDataFrame()
    i = 0
    message = 'Calculating centroids for ' + pop_type
    for point in tqdm(pop_cent, desc=message, position=0):
        if pop_type== "pop":
            pop = pop_data.iloc[i]['OverFifty']
            code = pop_data.iloc[i]['GEOID']
        if pop_type =="covid":
            pop = pop_data.iloc[i]['cases']
            code = pop_data.iloc[i].ZCTA5CE10
        pop_centroid = pop_centroid.append({'code':code,'pop': pop,'geometry': point}, ignore_index=True)
        i = i+1
    return(pop_centroid)

### Transform Population Data into Points

Use the pop_centroid function to:
1. convert at risk population data from Census Tract polygons into centroid points, and 
2. convert COVID case data from zip code tabulation area polygons into centroid points.

In [ ]:
%%time
pop_atrisk = pop_centroid(atrisk_data, "pop")
pop_covid = pop_centroid(covid_data, "covid")

## Analysis

The analysis section implements the enhanced two-step floating catchment area method in three phases. First, functions are defined to manage the computationally intensive analysis in a parallel processing framework. Second, the user may choose between two different populations (at-risk over 50 years, or COVID cases) and two different resource types (ICU beds or ventilators). Third, the functions and user selections are applied to to implement the E2SFCA method.

### calculate_catchment_area

Calculates a catchment area of things within some distance of a point using a given metric.

Function first creates an ego-centric subgraph on the NetworkX road network starting with the nearest OSM node for the hospital and going out to a given distance as measured by the distance unit. We then calculate the convex hull around the nodes in the ego-centric subgraph and make it a GeoPandas object.

Args:

* G: OSMNX network
* nearest_osm: OSMNX road network node that is closest to the place of interest (hospital)
* distance: the max distance to include in the catchment area
* distance_unit: how we measure distance (used by ego_graph), we always use time

Returns:

* GeoDataFrame the catchment area.

In [ ]:
def calculate_catchment_area(G, nearest_osm, distance, distance_unit = "time"):
    # Consutrct an ego graph based on distance unit for an input node
    road_network = nx.ego_graph(G, nearest_osm, distance, distance=distance_unit) 
    # Create point geometries for all nodes in ego graph
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    # Create a single part geometry of all nodes
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

### hospital_measure_acc

Measures the effect of a single hospital on the surrounding area. (Uses `calculate_catchment_area` or `djikstra_cca`)

Args:

* \_thread\_id: int used to keep track of which thread this is
* hospital: Geopandas dataframe with information on a hospital
* pop_data: Geopandas dataframe with population data
* distances: Distances in time to calculate accessibility for
* weights: how to weight the different travel distances

Returns:

* Tuple containing:
    * Int (\_thread\_id)
    * GeoDataFrame of catchment areas with key stats

In [ ]:
def hospital_measure_acc (_thread_id, hospital, pop_data, distances, weights):
    # Apply catchment calculation for each distance (10, 20, and 30 min)
    polygons = []
    for distance in distances:
        # Append djikstra catchment calculation (uncomment to use)
        polygons.append(calculate_catchment_area(G, hospital['nearest_osm'],distance))
    # Clip the overlapping distance ploygons (create two donuts + hole)
    for i in reversed(range(1, len(distances))):
        polygons[i] = gpd.overlay(polygons[i], polygons[i-1], how="difference")
    
    # Calculate accessibility measurements
    num_pops = []
    for j in pop_data.index:
        point = pop_data['geometry'][j]
        # Multiply polygons by weights
        for k in range(len(polygons)):
            if len(polygons[k]) > 0: # To exclude the weirdo (convex hull is not polygon)
                if (point.within(polygons[k].iloc[0]["geometry"])):
                    num_pops.append(pop_data['pop'][j]*weights[k])  
    total_pop = sum(num_pops)
    for i in range(len(distances)):
        polygons[i]['time']=distances[i]
        polygons[i]['total_pop']=total_pop
        polygons[i]['icu_beds'] = float(hospital['Adult ICU'])/polygons[i]['total_pop'] # proportion of # of beds over pops in 10 mins
        polygons[i]['vents'] = float(hospital['Total Vent'])/polygons[i]['total_pop'] # proportion of # of beds over pops in 10 mins
        polygons[i].crs = { 'init' : 'epsg:4326'}
        polygons[i] = polygons[i].to_crs({'init':'epsg:32616'})
    # print('\rCatchment for hospital {:4.0f} complete'.format(_thread_id), end=" ", flush=True)
    print('{:.0f}'.format(_thread_id), end=" ", flush=True)
    return(_thread_id, [ polygon.copy(deep=True) for polygon in polygons ])

### measure_acc_par

Parallel implementation of accessibility measurement.

Args:

* hospitals: Geodataframe of hospitals
* pop_data: Geodataframe containing population data
* network: OSMNX street network
* distances: list of distances to calculate catchments for
* weights: list of floats to apply to different catchments
* num\_proc: number of processors to use.

Returns:

* Geodataframe of catchments with accessibility statistics calculated

In [ ]:
def hospital_acc_unpacker(args):
    return hospital_measure_acc(*args)

# Parallel implementation of previous function
def measure_acc_par (hospitals, pop_data, network, distances, weights, num_proc = 4):
    catchments = []
    for distance in distances:
        catchments.append(gpd.GeoDataFrame())
    pool = mp.Pool(processes = num_proc)
    hospital_list = [ hospitals.iloc[i] for i in range(len(hospitals)) ]
    print("Calculating", len(hospital_list), "hospital catchments...\ncompleted number:", end=" ")
    results = pool.map(hospital_acc_unpacker, zip(range(len(hospital_list)), hospital_list, itertools.repeat(pop_data), itertools.repeat(distances), itertools.repeat(weights)))
    print("\nFinished calculating hospital catchments.")
    pool.close()
    results.sort()
    results = [ r[1] for r in results ]
    for i in range(len(results)):
        for j in range(len(distances)):
            catchments[j] = catchments[j].append(results[i][j], sort=False)
    return catchments

### overlap_calc

Calculates and aggregates accessibility statistics for one catchment on our grid file.

Args:

* \_id: thread ID
* poly: GeoDataFrame representing a catchment area
* grid_file: a GeoDataFrame representing our grids
* weight: the weight to applied for a given catchment
* service_type: the service we are calculating for: ICU beds or ventilators

Returns:

* Tuple containing:
    * thread ID
    * Counter object (dictionary for numbers) with aggregated stats by grid ID number

In [ ]:
from collections import Counter
def overlap_calc(_id, poly, grid_file, weight, service_type):
    value_dict = Counter()
    if type(poly.iloc[0][service_type])!=type(None):           
        value = float(poly[service_type])*weight
        # Find polygons that overlap hex grids
        intersect = gpd.overlay(grid_file, poly, how='intersection')
        # Get the intersection's area
        intersect['overlapped']= intersect.area
        # Divide overlapping area by total area to get percent
        intersect['percent'] = intersect['overlapped']/intersect['area']
        # Only choose intersecting catchments that make up greater than 50% of hexagon 
        intersect=intersect[intersect['percent']>=0.5]
        # Pull id
        intersect_region = intersect['id']
        for intersect_id in intersect_region:
            try:
                value_dict[intersect_id] +=value
            except:
                value_dict[intersect_id] = value
    return(_id, value_dict)

def overlap_calc_unpacker(args):
    return overlap_calc(*args)

### overlapping_function

Calculates how all catchment areas overlap with and affect the accessibility of each grid in our grid file.

Args:

* grid_file: GeoDataFrame of our grid
* catchments: GeoDataFrame of our catchments
* service_type: the kind of care being provided (ICU beds vs. ventilators)
* weights: the weight to apply to each service type
* num\_proc: the number of processors

Returns:

* Geodataframe - grid\_file with calculated stats

In [ ]:
def overlapping_function (grid_file, catchments, service_type, weights, num_proc = 4):
    grid_file[service_type]=0
    pool = mp.Pool(processes = num_proc)
    acc_list = []
    for i in range(len(catchments)):
        acc_list.extend([ catchments[i][j:j+1] for j in range(len(catchments[i])) ])
    acc_weights = []
    for i in range(len(catchments)):
        acc_weights.extend( [weights[i]]*len(catchments[i]) )
    results = pool.map(overlap_calc_unpacker, zip(range(len(acc_list)), acc_list, itertools.repeat(grid_file), acc_weights, itertools.repeat(service_type)))
    pool.close()
    results.sort()
    results = [ r[1] for r in results ]
    service_values = results[0]
    for result in results[1:]:
        service_values+=result
    for intersect_id, value in service_values.items():
        grid_file.loc[grid_file['id']==intersect_id, service_type] += value
    return(grid_file)

### normalization

Normalizes our result (Geodataframe) for a given resource (res).

In [ ]:
def normalization (result, res):
    result[res]=(result[res]-min(result[res]))/(max(result[res])-min(result[res]))
    return result

### Customize E2SFCA model parameters

Below you can customize the input of the model:

* Processor - the number of processors to use
* Population - the population to calculate the measure for
* Resource - the hospital resource of interest

In [ ]:
import ipywidgets
from IPython.display import display

processor_dropdown = ipywidgets.Dropdown( options=[("1", 1), ("2", 2), ("3", 3), ("4", 4)],
    value=4, description="Processor: ")

population_dropdown = ipywidgets.Dropdown( options=[("Population at Risk", "pop"), ("COVID-19 Patients", "covid") ],
    value="pop", description="Population: ")

resource_dropdown = ipywidgets.Dropdown( options=[("ICU Beds", "icu_beds"), ("Ventilators", "vents") ],
    value="icu_beds", description="Resource: ")

display(processor_dropdown,population_dropdown,resource_dropdown)

### Constant E2SFCA parameters

Define the distance bands (minutes of driving time) and weights 

In [ ]:
distances=[10, 20, 30] # Distances in travel time
weights=[1.0, 0.68, 0.22] # Weights where weights[0] is applied to distances[0]
print('distances: ', distances)
print('weights: ', weights)

### Calculate hospital catchment areas and service ratios

In [ ]:
%%time
if population_dropdown.value == "pop":
    catchments = measure_acc_par(hospitals, pop_atrisk, G, distances, weights, num_proc=processor_dropdown.value)
elif population_dropdown.value == "covid":
    catchments = measure_acc_par(hospitals, pop_covid, G, distances, weights, num_proc=processor_dropdown.value)

### Calculate local accessibility

In [ ]:
%%time
for j in range(len(catchments)):
    catchments[j] = catchments[j][catchments[j][resource_dropdown.value]!=float('inf')]
result = overlapping_function(grid_file, catchments, resource_dropdown.value, weights, num_proc=processor_dropdown.value)

In [ ]:
result[["id",resource_dropdown.value]].head()

In [ ]:
result = normalization(result, resource_dropdown.value)
result[["id",resource_dropdown.value]].head()

In [ ]:
# Save output to geopackage -- will name the layer according the dropdown parameters
result[["id",resource_dropdown.value,"geometry"]].to_file('data/derived/public/results.gpkg', 
                layer='{}_{}'.format(population_dropdown.value,resource_dropdown.value), 
                driver='GPKG')

## Results

### Histogram of Accessibility
Compare to figure 8 in the publication.

In [ ]:
# Make a histogram for the selected resource (ICU Beds or Ventilators)
if hasattr(result, resource_dropdown.value):
    result['icu_beds'].plot.hist(bins=10)
    plt.axvline(result['icu_beds'].mean(), color='k', linestyle='dashed', linewidth=1)
    plt.savefig('./results/figures/reproduction/{}_{}_histogram.png'.format(population_dropdown.value,resource_dropdown.value))
else:
    print(resource_dropdown.value, "not calculated yet.\n")

### Thematic Maps of Accessibility
Compare to figure 7 in the publication.

### Unclassified Accessibility Map

In [ ]:
def output_map(output_grid, hospitals, resource):
    ax=output_grid.plot(column=resource, 
                        cmap='PuBuGn',
                        figsize=(18,12), 
                        legend=True, 
                        zorder=1)
    # Next two lines set bounds for our x- and y-axes because it looks like there's a weird 
    # Point at the bottom left of the map that's messing up our frame (Maja)
    ax.set_xlim([325000, 370000])
    ax.set_ylim([550000, 600000])
    hospitals.plot(ax=ax, 
                   markersize=10, 
                   zorder=1, 
                   c='black', 
                   legend=False)

In [ ]:
hospitals = hospitals.to_crs({'init': 'epsg:26971'})
result = result.to_crs({'init': 'epsg:26971'})
output_map(result, hospitals, resource_dropdown.value)
# save as image with file name including the resource value, population value, and buffered / not buffered
plt.savefig('./results/figures/reproduction/{}_{}_buff{}_continuous.png'.format(population_dropdown.value, resource_dropdown.value, str(buffered)))

### Classified Accessibility Map

In [ ]:
def output_map_classified(output_grid, hospitals, resource):
    ax=output_grid.plot(column=resource, 
                        scheme='Equal_Interval', 
                        k=5, 
                        linewidth=0,
                        cmap='Blues', 
                        figsize=(18,12), 
                        legend=True, 
                        label="Acc Measure",
                        zorder=1)
    # Next two lines set bounds for our x- and y-axes because it looks like there's a weird 
    # Point at the bottom left of the map that's messing up our frame (Maja)
    ax.set_xlim([325000, 370000])
    ax.set_ylim([550000, 600000])
    hospitals.plot(ax=ax, 
                   markersize=10, 
                   zorder=2,
                   c='black',
                   legend=False,
                   )

In [ ]:
output_map_classified(result, hospitals, resource_dropdown.value)
# save as image with file name including the resource value, population value, and buffered / not buffered
plt.savefig('./results/figures/reproduction/{}_{}_buff{}_classified.png'.format(population_dropdown.value, resource_dropdown.value, str(buffered)))

## Compare Reproduction with Original Results

Load results provided with the original research compendium from shapefile `data/derived/public/Chicago_ACC.shp`. The file contains fields `hospital_i` for "ICU Beds" and `hospital_v` for "Ventilators". It is not known exactly which version of code was used to create this set of results, although it appears that a more complete road network was used (probably the full state of Illinois) than was provided with the compendium (Chicago only, without a buffer). It is not known whether the population was population at risk (>50 years old) or COVID patients (cases). However, the statistical and geographic distributions of each are extremly similar once the data has been normalized.

In [ ]:
# Import study results to compare
# hospital_i assumed to be for ICU and hospital_v assumed to be for ventilator
# however it's unknown whether the population is the COVID-19 population or the AT RISK population
fp = 'data/derived/public/Chicago_ACC.shp'
og_result = gpd.read_file(fp)
og_result.set_index("id")
og_result.head()

### Join original results to current results

In [ ]:
result.set_index("id")
result_compare = result.join(og_result[["hospital_i","hospital_v"]])
result_compare.head()

### Calculate Spearman's Rank Correlation

In [ ]:
# set original_resource variable to the name of original results column matching the modeling choice
if resource_dropdown.value == "icu_beds":
    original_resource = "hospital_i"
else:
    original_resource = "hospital_v"

# calculate spearman's rho
rho = stats.spearmanr(result_compare[[original_resource, resource_dropdown.value]])

# format text output
correlationmsg = "Comparing: \n" + "Original resource: " + resource_dropdown.value + " and population: unknown\n" + "Reproduction resource: " + resource_dropdown.value + " and population: " + population_dropdown.value
correlationmsg += "\nStreet network buffered? " + str(buffered)
print(correlationmsg)
"Rho = " + str(round(rho.correlation,3)) + ", pvalue = " + str(round(rho.pvalue,3))
print(rho_result)

### Create Scatterplot

In [ ]:
plt.scatter(result_compare[[original_resource]], result_compare[[resource_dropdown.value]], s=1)
plt.xlabel("Original", labelpad=5)
plt.ylabel("Reproduction", labelpad=5)
plt.text(.45, .08, rho_result, fontsize=8)
# save plot as image with file name including icu/ventilators and buffered or not
plt.savefig("./results/figures/reproduction/compare_{}_buffer_{}.png".format(resource_dropdown.value, str(buffered)))

## Conclusions

The reproduction results closely match the orginal results, with a Spearman's *Rho* close to 1 (perfect correlation). We can reject the NULL hypothesis of no correlation between the reproduction results and the original results. However, we also note several challenges and uncertainties. First, the large file sizes and processing times for the full state of Illinois caused challenges in using GitHub as the repository and scaling the reproduction efforts. This challenge was addressed by the original research team's decision to limit the reproduction study materials to the city of Chicago. Unfortunately, that decision led to unanticpated boundary effects and numerous bugs in the code. We have attempted to resolve all of those issues with this reproduction. Second, the original results were not provided with sufficient metadata to know which permutation of the data and code were used to generate them. They cannot be identically reproduced with the provided Chicago road network, they do not appear to identically match the maps in the published paper, and we have failed to identically them with a new buffered road network from OpenStreetMap. The new buffered network produces more similar results, with no discernable difference whether the at risk population (over the age of 50) or the COVID patients (cases) population has been used.

The reproduction effort has identified some additional sources of error and uncertainty which may be addressed with future work. *First*, there will always be boundary effects with the E2SFCA method, which can only be alleviated by including service points to a distance equal to the maximum travel time away from the study extent, and network data and population data to a distance equal to twice the maximum travel time away from the study extent. *Second*, there is substaintial missing data in OpenStreetMap speed limits, and the original study has applied speed limits of 35mph to all unknown road segments, even though many are highways and the non-highway urban speed limit in Illinois is 30mph. *Third*, the study has reaggregated population data to hospital catchments using centroid points, and has re-aggregated catchment areas to the final hexagonal grid if there is 50% or more overlap. Both reaggregations would be more accurate and less sensitive to the modifiable areal unit problem if they used area-weighted reaggregation. Finally, there are numerous opportunities to increase the computational efficiency of the code, e.g. by i) reducing redundancies between different versions of the model by processing and storing data before customizing model parameters, ii) reducing by a third the number of network analysis operations, and iii) using GeoPandas spatial indexing to optimize nearest neighbor searching and overlay analysis.

This reproduction study was feasible because the authors developed the research in an open science geospatial cyberinfrastructure community, CyberGISX. They published the majority of code required to execute the study, and made access to the computational infrastructure available to other researchers. This enabled the current research team to reproduce the study without building our own cyberinfrastructure, and with minimal or no prior knowledge using Jupyter notebooks or Python.